# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# Сгруппируем данные по столбцу с количеством детей и применим методы подсчета суммы, количества и вероятности задолженности 
# Таким образом мы получаем таблицу, в которой показано, сколько задолженностей у каждой группы и их общее количество
# Эта таблица нам понадобится для расчета вероятности задолженности по группам
data_grouped_children_debt = data.groupby('children').agg({'debt': ['sum', 'count', 'mean']})

In [23]:
# Выводим результат
data_grouped_children_debt

debt                 
           sum  count      mean
children                       
0         1063  14091  0.075438
1          444   4808  0.092346
2          194   2052  0.094542
3           27    330  0.081818
4            4     41  0.097561
5            0      9  0.000000

**Вывод: Взглянув на данные, можно понять что количество людей с 3,4 и 5 детьми недостаточно для формирования вывода по ним. Однако, взглянув на категории, имеющих 0, 1 и 2 детей, можно сказать, что чем больше детей, тем выше доля должников. Риск задолженности при наличии одного ребенка повышается на (9.23-7.54)/7.54 = 22.4%, а при наличии двух детей повышается на (9.45-7.54)/7.54 = 25.3%**

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
# Сгруппируем данные по столбцу с семейным статусом и применим методы подсчета суммы и количестваи и вероятности задолженности
# Таким образом мы получаем таблицу, в которой показано, сколько задолженностей у каждой группы и их общее количество
# Эта таблица нам понадобится для расчета вероятности задолженности по группам
data_grouped_family_status_debt = data.groupby('family_status').agg({'debt': ['sum', 'count', 'mean']})

In [26]:
# Выводим результат
data_grouped_family_status_debt

debt                 
                       sum  count      mean
family_status                              
Не женат / не замужем  273   2796  0.097639
в разводе               84   1189  0.070648
вдовец / вдова          63    951  0.066246
гражданский брак       385   4134  0.093130
женат / замужем        927  12261  0.075606

**Вывод: Наибольшая вероятность задолженностей холостых и состоящих в гражданском браке. Наименьшая у тех, кто был в браке, но лишился партнера. По статистике большинство людей не вошедших в брак имеют возраст до 25 лет. Судя по таблице эта возрастная категория имеет наивысший риск возникновения задолженности. Также по статистике абсолютное большинство переживают гибель партнера только после 40 лет. Самым «популярным» возрастом для разводящихся россиян является 30-34 года. Из этого можно сделать вывод, что взрослые люди старше 34 лет имеют меньше шансов к задолженности, чем молодые** 

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
# Сгруппируем данные по столбцу с категориями доходов и применим методы подсчета суммы, количества и вероятности задолженности
# Таким образом мы получаем таблицу, в которой показано, сколько задолженностей у каждой группы и их общее количество
# Эта таблица нам понадобится для расчета вероятности задолженности по группам
data_grouped_income_debt = data.groupby('total_income_category').agg({'debt': ['sum', 'count', 'mean']})

In [28]:
# Выводим результат
data_grouped_income_debt

debt                 
                        sum  count      mean
total_income_category                       
A                         2     25  0.080000
B                       354   5014  0.070602
C                      1353  15921  0.084982
D                        21    349  0.060172
E                         2     22  0.090909

**Вывод: Данных в категориях A, D, E не хватает для полноценного анализа, поэтому основываясь на категориях B и C можно сделать вывод что чем выше доход, тем ниже доля должников** 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [29]:
# Сгруппируем данные по столбцу с целями кредита и применим методы подсчета суммы, количества и вероятности задолженности
# Таким образом мы получаем таблицу, в которой показано, сколько задолженностей у каждой группы и их общее количество
# Эта таблица нам понадобится для расчета вероятности задолженности по группам
data_grouped_purpose_debt = data.groupby('purpose_category').agg({'debt': ['sum', 'count', 'mean']})

In [30]:
# Выводим результат
data_grouped_purpose_debt

debt                 
                          sum  count      mean
purpose_category                              
операции с автомобилем    400   4279  0.093480
операции с недвижимостью  780  10751  0.072551
получение образования     369   3988  0.092528
проведение свадьбы        183   2313  0.079118

**Вывод: Наибольшая вероятность задолженностей у людей, берущих кредит на операции с автомобилем. Наименьшая у тех, кто берет кредит для операций с недвижимостью.** 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ: Возможные причины возникновения пропусков:
1. Пропуски были в столбцах с ежемесячным доходом и опытом работы, откуда можно сдлать вывод, что пропуски были у людей, которые не работали раньше, и вместо 0 были подставлены значения Nan.
2. Клиенты могли не предоставить данные. Возможно, эту информацию не требуют в некоторых случаях.
3. Могли возникнуть проблемы  на этапах загрузки и выгрузки данных.
4. Неправильный запрос, объединение таблиц также может быть причиной.* 

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:  Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства.* 

### Шаг 4: общий вывод.

**Проведенная работа:**
1. Пропуски в столбцах с количество дней стажа и ежемесячным доходом были заменены медианным значением.
2. Были удалены аномальные значения по количеству детей.
3. Были удалены дубликаты данных
4. Был создан новый столбец, обобщяющий группы по размеру ежемесячного дохода: A, B, C, D, E.
5. Был создан новый столбец, обобщяющий причины взятия кредита в котором все имеющиеся причины распередляются в одну из 4 новых категорий.
6. Был проведен анализ в котором искалась зависимость между риском возникновения задолженности и: количеством детей, семейным положением, уровнем дохода и целью взятия кредита.
    
**Вывод:**

1. По людям с 0-2 детьми: чем больше детей, тем выше доля должников. Данных по многодетным (3 и более детей) недостаточно для корректных выводов по ним. 
2. По семейному положению: люди, побывавшие в официальном браке, наиболее надежны. 
3. По группам дохода B и C: чем выше доход, тем ниже доля должников. 
4. По цели кредита: наиболее рискованные категории - 'операции с автомобилем' и 'получение образования', наиболее надежные - 'операции с недвижимостью' и 'проведение свадьбы'.

**Рекомандации:**

В качестве рекомендаций можно предложить другую категоризацию по доходам. Можно предложить исследовать другие признаки, рассчитать другие метрики (отношение дохода к сумме выплат и др.). В анкете сделать поля о доходе и стаже обязательными к заполнению, добавить автопроверку на формат вводимых пользователем данных